In [1]:
# Install necessary packages
!pip install pandas scikit-learn joblib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Set display options for pandas
pd.set_option('display.max_column', 50)

# Load the best model and the scaler
best_rf_model = joblib.load('models/random_forest_model.pkl')
scaler = joblib.load('models/scaler.pkl')

# Load the new test data
test_data_path = 'dataset/test_data.csv'
data = pd.read_csv(test_data_path)

# List of columns to keep (same as training data columns)
columns_to_keep = [
    'Application mode', 'Application order', 'Course', 
    'Previous qualification (grade)', 'Mother\'s qualification', 'Father\'s qualification', 
    'Mother\'s occupation', 'Father\'s occupation', 'Admission grade', 'Debtor', 
    'Tuition fees up to date', 'Scholarship holder', 'Age at enrollment', 
    'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 
    'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 
    'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 
    'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 
    'Unemployment rate', 'Inflation rate', 'GDP'
]

# Keep only the specified columns in the data
data = data[columns_to_keep]

In [3]:
# Step 1: Encode categorical columns (if any)
# Use LabelEncoder for each categorical column that the model may expect
categorical_columns = ['Application mode', 'Course', 'Previous qualification (grade)', 
                       'Mother\'s qualification', 'Father\'s qualification', 
                       'Mother\'s occupation', 'Father\'s occupation', 
                       'Debtor', 'Tuition fees up to date', 'Scholarship holder']

# Apply LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))  # Encoding as string, to handle any unseen values
    label_encoders[col] = le  # Save the label encoder for future use

In [4]:
# Step 2: Scale numerical data
# Scale the data using the same scaler that was used during model training
data_scaled = scaler.transform(data)

In [5]:
# Step 3: Make predictions using the trained model
predictions = best_rf_model.predict(data_scaled)

In [6]:
# Step 4: Map predictions (assuming 0 = dropout, 1 = graduate)
prediction_labels = ['Dropout', 'Graduate']
predicted_outcomes = [prediction_labels[pred] for pred in predictions]

# Add predictions to the data
data['Prediction'] = predicted_outcomes

In [7]:
# Save the result to a CSV file
output_path = 'dataset/prediction.csv'
data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

Predictions saved to dataset/prediction.csv


In [8]:
# Print total count of Graduate and Dropout predictions
prediction_counts = data['Prediction'].value_counts()
print(f"Total Graduate: {prediction_counts.get('Graduate', 0)}")
print(f"Total Dropout: {prediction_counts.get('Dropout', 0)}")

Total Graduate: 2730
Total Dropout: 1694


In [9]:
data.describe()

,Application mode,Application order,Course,Previous qualification (grade),Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Admission grade,Debtor,Tuition fees up to date,Scholarship holder,Age at enrollment,Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,GDP
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,4.992315,1.727848,8.853300,38.951175,9.839060,13.466546,24.516501,35.336347,126.978119,0.113698,0.880651,0.248418,23.265145,6.270570,8.299051,4.706600,10.640822,6.232143,8.063291,4.435805,10.230206,11.566139,1.228029,0.001969
std,4.981624,1.313793,4.419649,17.354827,7.260213,9.353207,6.692848,13.063382,14.482001,0.317480,0.324235,0.432144,7.587816,2.480178,4.179106,3.094238,4.843663,2.195951,3.947951,3.014764,5.210808,2.663850,1.382711,2.269935
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,0.000000,1.000000,5.000000,29.000000,3.000000,7.000000,24.000000,37.000000,117.900000,0.000000,1.000000,0.000000,19.000000,5.000000,6.000000,3.000000,11.000000,5.000000,6.000000,2.000000,10.750000,9.400000,0.300000,-1.700000
50%,4.000000,1.000000,9.000000,39.000000,12.000000,15.000000,25.000000,40.000000,126.100000,0.000000,1.000000,0.000000,20.000000,6.000000,8.000000,5.000000,12.285714,6.000000,8.000000,5.000000,12.200000,11.100000,1.400000,0.320000
75%,9.000000,2.000000,12.000000,50.000000,17.000000,22.000000,29.000000,42.000000,134.800000,0.000000,1.000000,0.000000,25.000000,7.000000,10.000000,6.000000,13.400000,7.000000,10.000000,6.000000,13.333333,13.900000,2.600000,1.790000
max,17.000000,9.000000,16.000000,100.000000,28.000000,33.000000,31.000000,45.000000,190.000000,1.000000,1.000000,1.000000,70.000000,26.000000,45.000000,26.000000,18.875000,23.000000,33.000000,20.000000,18.571429,16.200000,3.700000,3.510000
